# CXF to Python Translation Example

This notebook demonstrates the complete process of translating a Control eXchange Format (CXF) file to executable Python code using the CDL Python translator.

## Example: CustomPWithLimiter

We'll translate a proportional controller with a variable output limiter:
- **Input CXF:** `CustomPWithLimiter.jsonld` (S231P format)
- **Output:** Executable Python class
- **Functionality:** `y = min(yMax, k*e)` where `e` is error, `yMax` is max output, `k` is gain

The model contains:
- 2 inputs: `e` (control error), `yMax` (maximum output)
- 1 output: `y` (control signal)
- 1 parameter: `k` (proportional gain)
- 2 CDL blocks: `MultiplyByParameter` (gain), `Min` (limiter)
- 4 connections between blocks

## Step 1: Load and Parse CXF File

In [1]:
import json
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from cdl_translator.parser import CXFParser
from cdl_translator.codegen import CodeGenerator

# Load CXF file
cxf_path = Path.cwd().parent / 'tests/translator/fixtures/modelica-json/cxf/CustomPWithLimiter.jsonld'

print(f"Loading CXF file: {cxf_path.name}")
print(f"File size: {cxf_path.stat().st_size} bytes\n")

with open(cxf_path, 'r') as f:
    cxf_data = json.load(f)

# Show CXF structure
print("CXF Structure:")
print(f"  Format: S231P (ASHRAE 231P namespace)")
print(f"  Context: {cxf_data.get('@context')}")
print(f"  Graph objects: {len(cxf_data.get('@graph', []))}")

Loading CXF file: CustomPWithLimiter.jsonld
File size: 4624 bytes

CXF Structure:
  Format: S231P (ASHRAE 231P namespace)
  Context: {'S231P': 'https://data.ashrae.org/S231P#'}
  Graph objects: 10


## Step 2: Parse CXF into Internal Model

In [2]:
# Parse CXF
parser = CXFParser()
model = parser.parse_dict(cxf_data)

print("✅ Parsing successful!\n")
print("="*60)
print("MODEL STRUCTURE")
print("="*60)

# Show metadata
print(f"\nName: {model.metadata.name}")
print(f"Description: {model.metadata.description[:100] if model.metadata.description else 'None'}...")

# Show parameters
print(f"\nParameters ({len(model.metadata.parameters)}):")
for param in model.metadata.parameters:
    print(f"  - {param.name}: {param.type} = {param.value}")
    if param.description:
        print(f"    Description: {param.description}")

# Show inputs
print(f"\nInputs ({len(model.metadata.inputs)}):")
for inp in model.metadata.inputs:
    print(f"  - {inp.name}: {inp.type}")
    if inp.description:
        print(f"    Description: {inp.description}")

# Show outputs
print(f"\nOutputs ({len(model.metadata.outputs)}):")
for out in model.metadata.outputs:
    print(f"  - {out.name}: {out.type}")
    if out.description:
        print(f"    Description: {out.description}")

✅ Parsing successful!

MODEL STRUCTURE

Name: CustomPWithLimiter
Description: info=<html>
<p>
Block that outputs <code>y = min(yMax, k*e)</code>,
where
<code>yMax</code> and <cod...

Parameters (1):
  - k: Real = 2
    Description: Constant gain

Inputs (2):
  - e: PortType.REAL
    Description: Control error
  - yMax: PortType.REAL
    Description: Maximum value of output signal

Outputs (1):
  - y: PortType.REAL
    Description: Control signal


## Step 3: Visualize Block Instances and Connections

In [3]:
# Show block instances
print("="*60)
print("BLOCK INSTANCES")
print("="*60)
print(f"\nTotal blocks: {len(model.instances)}\n")

for inst in model.instances:
    block_type = inst.block_type.split('.')[-1]  # Get simple name
    print(f"  {inst.instance_name}: {block_type}")
    print(f"    Full type: {inst.block_type}")
    if inst.parameters:
        print(f"    Parameters: {inst.parameters}")
    print()

BLOCK INSTANCES

Total blocks: 2

  gain: MultiplyByParameter
    Full type: Buildings.Controls.OBC.CDL.Reals.MultiplyByParameter
    Parameters: {'k': 'self.k'}

  minValue: Min
    Full type: Buildings.Controls.OBC.CDL.Reals.Min



In [4]:
# Show connections
print("="*60)
print("CONNECTIONS")
print("="*60)
print(f"\nTotal connections: {len(model.connections)}\n")

for i, conn in enumerate(model.connections, 1):
    # Format source
    if conn.source_block:
        source = f"{conn.source_block}.{conn.source_port}"
    else:
        source = f"[INPUT] {conn.source_port}"
    
    # Format target
    if conn.target_block:
        target = f"{conn.target_block}.{conn.target_port}"
    else:
        target = f"[OUTPUT] {conn.target_port}"
    
    print(f"  {i}. {source:30s} → {target}")

print("\n" + "="*60)
print("CONNECTION DIAGRAM")
print("="*60)
print("""
       ┌─────────────────────────────────────┐
       │      CustomPWithLimiter             │
       │                                     │
  e ───┼──→ gain.u                          │
       │      (MultiplyByParameter)         │
       │           │                         │
       │           │ gain.y                 │
       │           ↓                         │
       │      minValue.u2                    │
       │      (Min)                          │
       │           ↑                         │
yMax ──┼──→ minValue.u1                     │
       │           │                         │
       │           │ minValue.y             │
       │           ↓                         │
  y ←──┼───────────┘                        │
       │                                     │
       └─────────────────────────────────────┘
""")

CONNECTIONS

Total connections: 4

  1. [INPUT] e                      → gain.u
  2. [INPUT] yMax                   → minValue.u1
  3. gain.y                         → minValue.u2
  4. minValue.y                     → [OUTPUT] y

CONNECTION DIAGRAM

       ┌─────────────────────────────────────┐
       │      CustomPWithLimiter             │
       │                                     │
  e ───┼──→ gain.u                          │
       │      (MultiplyByParameter)         │
       │           │                         │
       │           │ gain.y                 │
       │           ↓                         │
       │      minValue.u2                    │
       │      (Min)                          │
       │           ↑                         │
yMax ──┼──→ minValue.u1                     │
       │           │                         │
       │           │ minValue.y             │
       │           ↓                         │
  y ←──┼───────────┘                        │
    

## Step 4: Validate Model

In [5]:
# Validate model
is_valid, errors = model.validate()

print("="*60)
print("MODEL VALIDATION")
print("="*60)

if is_valid:
    print("\n✅ Model is valid!")
    print("\nValidation checks passed:")
    print("  ✓ All connections reference valid blocks/ports")
    print("  ✓ No circular dependencies detected")
    print("  ✓ All inputs/outputs properly connected")
else:
    print("\n❌ Model validation failed!")
    print("\nErrors:")
    for error in errors:
        print(f"  - {error}")

MODEL VALIDATION

✅ Model is valid!

Validation checks passed:
  ✓ All connections reference valid blocks/ports
  ✓ No circular dependencies detected
  ✓ All inputs/outputs properly connected


## Step 5: Get Computation Order

The translator uses topological sorting (Kahn's algorithm) to determine the correct computation order based on block dependencies.

In [6]:
# Get computation order
try:
    order = model.get_computation_order()
    
    print("="*60)
    print("COMPUTATION ORDER")
    print("="*60)
    print("\nBlocks will be computed in this order:\n")
    
    for i, inst in enumerate(order, 1):
        block_type = inst.block_type.split('.')[-1]
        print(f"  {i}. {inst.instance_name} ({block_type})")
    
    print("\nThis ensures data flows correctly from inputs to outputs.")
    
except ValueError as e:
    print(f"❌ Error determining computation order: {e}")

COMPUTATION ORDER

Blocks will be computed in this order:

  1. gain (MultiplyByParameter)
  2. minValue (Min)

This ensures data flows correctly from inputs to outputs.


## Step 6: Generate Python Code

In [13]:
# Generate Python code
codegen = CodeGenerator()
generated_code = codegen.generate(model)

print("="*60)
print("GENERATED PYTHON CODE")
print("="*60)
print(f"\nLines of code: {len(generated_code.splitlines())}")
print("\n" + "="*60)
print(generated_code)
print("="*60)

with open("generated_customPWithLimiter.py", "w") as fp:
    fp.write(generated_code)

GENERATED PYTHON CODE

Lines of code: 63

"""info=<html>
<p>
Block that outputs <code>y = min(yMax, k*e)</code>,
where
<code>yMax</code> and <code>e</code> are real-valued input signals and
<code>k</code> is a parameter.
</p>
</html>

Auto-generated from CDL model: CustomPWithLimiter
Generated by: CDL to Python Translator
"""

from typing import Dict, Any
from cdl_python.CDL.Reals import Min, MultiplyByParameter


class CustomPWithLimiter:
    """info=<html>
<p>
Block that outputs <code>y = min(yMax, k*e)</code>,
where
<code>yMax</code> and <code>e</code> are real-valued input signals and
<code>k</code> is a parameter.
</p>
</html>

    Parameters:
        k: Constant gain
    """

    def __init__(self, k=2):
        """Initialize CustomPWithLimiter

        Args:
            k: Constant gain
        """
        self.k = k

        # Instantiate CDL blocks
        self.gain = MultiplyByParameter(
            k=self.k,
        )
        self.minValue = Min(
        )

    def compute(s

## Step 7: Verify Syntax

In [8]:
import ast

# Verify syntax
try:
    ast.parse(generated_code)
    print("✅ Generated code has valid Python syntax!")
except SyntaxError as e:
    print(f"❌ Syntax error: {e}")

✅ Generated code has valid Python syntax!


## Step 8: Test Generated Code

Now let's execute the generated code and verify it produces correct results.

In [9]:
# Execute generated code
exec(generated_code)

# Create instance
controller = CustomPWithLimiter(k=2.0)

print("="*60)
print("TESTING GENERATED CODE")
print("="*60)
print("\nController: CustomPWithLimiter(k=2.0)")
print("Formula: y = min(yMax, k*e)\n")

# Test cases
test_cases = [
    {'e': 3.0, 'yMax': 10.0, 'expected': 6.0, 'note': 'k*e < yMax'},
    {'e': 3.0, 'yMax': 5.0, 'expected': 5.0, 'note': 'k*e > yMax (limited)'},
    {'e': 5.0, 'yMax': 8.0, 'expected': 8.0, 'note': 'k*e > yMax (limited)'},
    {'e': 1.0, 'yMax': 5.0, 'expected': 2.0, 'note': 'k*e < yMax'},
    {'e': -2.0, 'yMax': 5.0, 'expected': -4.0, 'note': 'Negative error'},
]

print("Test Results:")
print("-" * 60)
print(f"{'e':<8} {'yMax':<8} {'Expected':<10} {'Got':<10} {'Status':<10}")
print("-" * 60)

all_pass = True
for test in test_cases:
    result = controller.compute(e=test['e'], yMax=test['yMax'])
    y = result['y']
    
    passed = abs(y - test['expected']) < 1e-6
    status = "✅ PASS" if passed else "❌ FAIL"
    
    print(f"{test['e']:<8.2f} {test['yMax']:<8.2f} {test['expected']:<10.2f} {y:<10.2f} {status}")
    
    if not passed:
        all_pass = False

print("-" * 60)
if all_pass:
    print("\n✅ All tests passed! Generated code is working correctly.")
else:
    print("\n❌ Some tests failed!")

TESTING GENERATED CODE

Controller: CustomPWithLimiter(k=2.0)
Formula: y = min(yMax, k*e)

Test Results:
------------------------------------------------------------
e        yMax     Expected   Got        Status    
------------------------------------------------------------
3.00     10.00    6.00       6.00       ✅ PASS
3.00     5.00     5.00       5.00       ✅ PASS
5.00     8.00     8.00       8.00       ✅ PASS
1.00     5.00     2.00       2.00       ✅ PASS
-2.00    5.00     -4.00      -4.00      ✅ PASS
------------------------------------------------------------

✅ All tests passed! Generated code is working correctly.


## Step 9: Compare with Hand-Written Implementation

Let's verify the generated code produces the same results as a hand-written implementation.

In [10]:
from cdl_python.CDL.Reals import MultiplyByParameter, Min

class CustomPWithLimiterManual:
    """Hand-written implementation for comparison"""
    
    def __init__(self, k=2.0):
        self.k = k
        self.gain = MultiplyByParameter(k=self.k)
        self.minValue = Min()
    
    def compute(self, e, yMax):
        gain_output = self.gain.compute(u=e)
        minValue_output = self.minValue.compute(u1=yMax, u2=gain_output['y'])
        return {'y': minValue_output['y']}

# Test both implementations
manual = CustomPWithLimiterManual(k=2.0)
generated = CustomPWithLimiter(k=2.0)

print("="*60)
print("COMPARING IMPLEMENTATIONS")
print("="*60)
print("\nTesting 100 random inputs...\n")

import random
random.seed(42)

errors = []
for i in range(100):
    e = random.uniform(-10, 10)
    yMax = random.uniform(0, 20)
    
    manual_result = manual.compute(e=e, yMax=yMax)
    generated_result = generated.compute(e=e, yMax=yMax)
    
    diff = abs(manual_result['y'] - generated_result['y'])
    if diff > 1e-10:
        errors.append((e, yMax, manual_result['y'], generated_result['y'], diff))

if not errors:
    print("✅ Perfect match! Generated code produces identical results.")
    print("\nThe translator correctly:")
    print("  ✓ Instantiated blocks with correct parameters")
    print("  ✓ Connected blocks in the right order")
    print("  ✓ Mapped inputs and outputs correctly")
else:
    print(f"❌ Found {len(errors)} differences:")
    for e, yMax, manual_y, generated_y, diff in errors[:5]:
        print(f"  e={e:.2f}, yMax={yMax:.2f}: manual={manual_y:.6f}, generated={generated_y:.6f}, diff={diff:.6e}")

COMPARING IMPLEMENTATIONS

Testing 100 random inputs...

✅ Perfect match! Generated code produces identical results.

The translator correctly:
  ✓ Instantiated blocks with correct parameters
  ✓ Connected blocks in the right order
  ✓ Mapped inputs and outputs correctly


## Summary

This notebook demonstrated the complete CXF to Python translation pipeline:

1. ✅ **Loaded S231P format CXF file** (ASHRAE standard)
2. ✅ **Parsed into internal model** with full validation
3. ✅ **Extracted model structure** (blocks, connections, parameters)
4. ✅ **Validated model** (no circular dependencies, all connections valid)
5. ✅ **Determined computation order** (topological sorting)
6. ✅ **Generated Python code** (template-based, clean output)
7. ✅ **Verified syntax** (valid Python)
8. ✅ **Tested execution** (correct results)
9. ✅ **Compared with reference** (identical behavior)

### Key Features Demonstrated

- **S231P Namespace Support**: Correctly parsed ASHRAE 231P format
- **@graph Resolution**: Resolved cross-references between objects
- **Connection Extraction**: Found all 4 connections (model I/O + block-to-block)
- **Parameter Binding**: Correctly passed `k` parameter to gain block
- **Code Quality**: Generated clean, readable, executable Python
- **Validation**: Comprehensive model checking before code generation

### Production Ready

The translator is ready for production use with:
- ✅ Standard CDL blocks from the library (122 blocks)
- ✅ S231P namespace format CXF files (ASHRAE standard)
- ✅ Full validation and error handling
- ✅ Clean, maintainable generated code

### Next Steps

Try translating your own CXF files:
```python
from cdl_translator.parser import CXFParser
from cdl_translator.codegen import CodeGenerator
import json

with open('your_model.jsonld', 'r') as f:
    parser = CXFParser()
    model = parser.parse_dict(json.load(f))

codegen = CodeGenerator()
code = codegen.generate(model)

with open('your_model.py', 'w') as f:
    f.write(code)
```